In [ ]:
import os
import pandas as pd
import datetime
import tensorflow as tf
import seaborn
import numpy as np
from scipy import stats
from matplotlib import pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.preprocessing import StandardScaler
import math
from sklearn.model_selection import TimeSeriesSplit

In [ ]:
df = pd.read_excel("M3C.xls", usecols="A:Z")
df_micro = df.iloc[0:146,]
df_micro = df_micro.iloc[:,6:32]
df_micro_clean = df_micro.T
df_micro_clean

,0,1,2,3,4,5,6,7,8,9,...,136,137,138,139,140,141,142,143,144,145
1,940.66,1991.05,1461.57,744.54,4977.18,1458.05,2399.26,420.06,1325.42,2160.04,...,2871.6,5208.5,2514.0,543.90,1494.15,5221.5,1466.0,1198.0,1109.31,963.45
2,1084.86,2306.40,1692.50,1105.16,5248.00,1931.53,2910.52,423.66,1801.76,2553.48,...,3145.8,5420.5,2167.0,439.05,1706.03,5299.0,1764.0,1427.5,1228.26,1234.95
3,1244.98,2604.00,2193.82,1417.40,5370.00,2315.78,3126.62,476.43,1996.05,2901.18,...,3352.2,5365.0,2231.0,370.95,1984.73,5066.0,1986.0,1748.0,1235.82,1841.00
4,1445.02,2992.30,2459.68,1838.04,6184.89,2573.11,3475.14,576.81,2865.77,3205.21,...,3643.8,5526.5,2954.0,728.40,2358.75,5229.0,2238.0,2028.5,1350.42,2775.50
5,1683.17,3722.08,3246.80,2337.62,7137.19,2916.61,3750.96,743.19,3543.31,4059.58,...,4255.0,5493.0,2221.0,1137.60,2816.85,4124.5,2374.0,2435.0,1528.77,3402.15
6,2038.15,5226.62,4748.86,3094.88,6743.00,3018.94,3752.72,881.73,4826.29,4742.69,...,5358.4,5434.0,750.0,1519.05,3264.75,5278.0,2418.0,1993.0,1411.20,2895.55
7,2342.52,5989.46,5559.46,4280.04,7298.00,3107.36,4004.02,1345.29,6073.39,5063.38,...,6453.8,5171.5,1024.0,1030.50,3717.45,5650.5,2634.0,2458.0,1366.05,2151.30
8,2602.45,5614.62,5292.42,5070.20,5260.29,2912.45,3737.38,1371.00,7335.71,6154.50,...,2599.4,4637.0,1766.0,655.20,4201.35,5586.5,4290.0,2036.0,1569.54,2321.55
9,2927.87,5527.00,5029.40,3675.18,4898.18,3450.89,4263.98,1551.63,5300.43,7146.27,...,123.6,4178.5,4107.0,1301.40,4796.03,4733.5,4736.0,2321.5,1658.07,2195.70
10,3103.96,5389.80,4753.60,3667.38,5392.04,3816.85,5121.08,3146.37,4401.13,7839.00,...,4027.8,4303.5,5386.0,1791.45,5224.88,3856.5,5598.0,3472.5,1666.53,3012.55


In [ ]:
MEAN = df_micro_clean.mean()
STD = df_micro_clean.std()

In [ ]:
# De-trend (each time series at a time)
#x = df_micro_clean.astype(str).astype(float)
#y = df_micro_clean.index.astype(str).astype(int)
#x.iloc[:, 0].to_numpy()

#for i in x:
  #poly_fit = np.polyfit(y, x.iloc[:, i].to_numpy(), 1)
  # divide by trend
  #df_micro_clean[i] = df_micro_clean[i] / (np.exp(poly_fit[i] * x.index + poly_fit[1]))



In [ ]:
# Standardising
scaler = StandardScaler()
df_micro_clean = scaler.fit_transform(df_micro_clean)
df_micro_clean = pd.DataFrame(df_micro_clean)

In [ ]:
def get_labelled_window(x, horizon=1):
  return x[:, :-horizon], x[:, -horizon]

def make_windows(x, window_size=4, horizon=1):
  window_step = np.expand_dims(np.arange(window_size+horizon), axis=0)
  window_indexes = window_step + np.expand_dims(np.arange(len(x)-(window_size+horizon-1)), axis=0).T # create 2D array of windows of window size
  #print(f"Window indexex:\n {window_indexes, window_indexes.shape}")
  windowed_array = x[window_indexes]
  windows, labels = get_labelled_window(windowed_array, horizon=horizon)
  return windows, labels

In [ ]:
data_split = []
for j in range(20):
  test = np.array(df_micro_clean.iloc[j, :])
  data_split.append(test)

full_windows, full_labels = make_windows(np.array(data_split))

In [ ]:
columns = df_micro_clean.columns
series_length = int(len(full_windows))

train_size =  math.ceil(series_length * 0.70)
val_size = math.floor(series_length * 0.15)
test_size = math.floor(series_length * 0.15)

train_x = full_windows[0:train_size]
train_y = full_labels[0:train_size]

val_x = full_windows[train_size:(series_length - val_size)]
val_y = full_labels[train_size:(series_length - val_size)]

test_x = full_windows[(series_length - val_size):series_length]
test_y = full_labels[(series_length - val_size):series_length]

In [ ]:
# Create a function to implement a ModelCheckpoint callback
def create_model_checkpoint(model_name, save_path="model_experiments"):
  return tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(save_path, model_name),
                                            verbose=0,
                                            save_best_only=True)

In [ ]:
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasRegressor
tf.random.set_seed(42)

model = tf.keras.Sequential([
    tf.keras.layers.Reshape((4*146,), input_shape=(4, 146)),
    tf.keras.layers.Dense(256, activation="relu", kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    tf.keras.layers.Dense(256, activation="relu", kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    tf.keras.layers.Dense(256, activation="relu", kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    tf.keras.layers.Dense(146, activation="linear") 
], name="model_2_dense")

model.compile(loss="mse",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["mse", "mae"]) 

eval_scores = []

# SMAPE
def smape(y_true, y_pred):
    return 200 * np.mean(np.abs(y_pred - y_true) / (np.abs(y_pred) + np.abs(y_true)))

tscv = TimeSeriesSplit(n_splits=11)

learning_rates = [0.001, 0.01, 0.1]
batch_sizes = [16, 32, 64]
regularizations = [0.001, 0.01, 0.1]

best_smape = float('inf')
best_hyperparameters = {}

for train_index, test_index in tscv.split(train_x):
    train_x_cv, test_x_cv = train_x[train_index], train_x[test_index]
    train_y_cv, test_y_cv = train_y[train_index], train_y[test_index]
    
    for learning_rate in learning_rates:
        for batch_size in batch_sizes:
            for regularization in regularizations:
                # Create model with selected hyperparameters
                model = tf.keras.Sequential([
                    tf.keras.layers.Reshape((4*146,), input_shape=(4, 146)),
                    tf.keras.layers.Dense(11, activation="relu", kernel_regularizer=tf.keras.regularizers.l2(regularization)),
                    tf.keras.layers.Dense(11, activation="relu", kernel_regularizer=tf.keras.regularizers.l2(regularization)),
                    tf.keras.layers.Dense(11, activation="relu", kernel_regularizer=tf.keras.regularizers.l2(regularization)),
                    tf.keras.layers.Dense(146, activation="linear") 
                ], name="model")

                model.compile(loss="mse",
                              optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                              metrics=["mse", "mae"])

                model.fit(train_x_cv, train_y_cv, epochs=30, batch_size=batch_size, verbose=0)
                predictions = model.predict(test_x_cv)
                smape_score = smape(test_y_cv, predictions)
                
                if smape_score < best_smape:
                    best_smape = smape_score
                    best_hyperparameters = {
                        'learning_rate': learning_rate,
                        'batch_size': batch_size,
                        'regularization': regularization
                    }

print("Best Hyperparameters:", best_hyperparameters)
print("Best SMAPE Score:", best_smape)

1/1 [==============================] - 0s 90ms/step
Best Hyperparameters: {'learning_rate': 0.01, 'batch_size': 16, 'regularization': 0.01}
Best SMAPE Score: 50.78240969383096


In [ ]:
# Train the model on the full training dataset
model.fit(train_x, train_y, epochs=30, validation_data=(val_x, val_y), batch_size=32, verbose=1, callbacks=[create_model_checkpoint(model_name=model.name)])

In [ ]:
data = [[[ 5.57144841e-01,  2.70844688e-01, -4.08019395e-01,
         6.60263272e-01, -3.74639409e-02,  1.24919175e+00,
         9.25597972e-01,  1.45025912e+00,  6.12451831e-01,
         1.43892449e+00,  1.10022222e+00,  5.69950131e-01,
         1.02618087e+00,  8.43926863e-01,  9.02372273e-01,
         1.25224131e+00,  1.21374656e+00,  9.60775488e-01,
         8.93938224e-01,  1.43484517e+00,  8.59203901e-01,
         1.46012644e+00, -3.48134869e-01,  9.27750672e-01,
         7.84352246e-01,  1.49215734e+00,  3.97543985e-01,
         1.45163241e+00,  2.83084088e-01,  8.66736731e-01,
         9.02748510e-01,  1.09023288e+00,  1.65788607e+00,
         2.83014805e-01, -4.08019395e-01,  1.58413061e-01,
         3.98803421e-01,  8.49831358e-01,  5.05287264e-01,
         4.17472426e-01,  1.75235319e+00,  8.58247629e-01,
         1.10652455e+00,  9.15800763e-01,  6.65456806e-01,
         8.49727641e-01,  3.74475201e-01,  7.82243460e-01,
         9.20871702e-01,  1.56600499e+00, -1.08600195e-01,
         8.01563832e-01,  1.57891978e+00,  6.29813892e-01,
         2.93176960e-03,  1.10374708e+00,  6.97041786e-01,
         3.06224658e-01,  3.68093119e-01,  8.05442036e-01,
         6.59002712e-01,  9.60468195e-01,  1.01384064e+00,
         4.99861556e-01,  4.77570404e-01,  1.05266434e+00,
         5.45066173e-01,  1.74121674e-01, -1.44434329e-01,
         8.98074706e-01,  7.96948751e-01,  1.00143043e+00,
         1.13427068e+00, -1.99984778e-01,  1.73562457e+00,
         6.14079983e-01,  7.18662610e-01,  8.52246326e-01,
         1.39150612e+00,  3.96978088e-01,  1.24870395e+00,
         8.00722239e-01,  3.74012336e-01,  1.64124806e+00,
         7.64711055e-01,  1.28558726e+00, -4.17623356e-01,
         8.93322333e-01,  6.33149039e-01,  1.06788148e+00,
         5.06497562e-01,  5.04046321e-01,  4.46150404e-01,
         1.21339825e+00,  1.45937997e-01,  3.74030426e-01,
         6.50666192e-01,  1.17319251e+00, -1.84520379e-01,
        -1.02568836e-01,  5.28678842e-02,  1.18037686e+00,
         1.70358831e-02,  8.39121954e-01,  8.12713047e-01,
         1.19595956e+00, -4.27039641e-01,  1.79336364e-01,
        -8.63533503e-01,  1.20071030e+00,  2.80865555e-01,
         1.32281869e+00,  2.39249870e+00,  1.03853483e+00,
         7.11500320e-01, -4.15872196e-01,  9.73740332e-01,
         7.01764514e-01,  2.47666517e-01,  1.27376846e+00,
         3.27246732e-01,  1.25114589e+00,  5.69324373e-01,
         9.85506388e-01,  6.99242729e-01,  5.32245705e-01,
         2.26236299e+00,  2.22355017e+00,  1.27674939e+00,
        -1.28968282e+00,  4.91035418e-01,  8.14690030e-01,
         1.12967874e+00,  1.32663888e+00,  1.17056909e+00,
         1.99880129e-01,  1.33898339e-01, -1.15703803e+00,
         6.39937570e-02,  4.54669565e-01,  1.22590148e+00,
         1.21719025e+00,  9.72102967e-01,  5.49377698e-01,
         5.53766211e-01,  5.13854094e-02],
       [ 8.68710377e-01,  9.55161919e-01,  6.94395477e-02,
         1.04885026e+00, -4.31184878e-01,  1.05507829e+00,
         8.43238267e-01,  1.49392817e+00,  1.18978349e+00,
         1.48549383e+00,  1.33312580e+00,  7.70345927e-01,
         1.36455045e+00,  9.64161648e-01,  1.17863389e+00,
         1.65198550e+00,  1.16275169e+00,  8.23130417e-01,
         5.67597237e-01,  1.43850523e+00,  1.19358491e+00,
         1.04906384e+00, -1.72254879e-01,  1.03590766e+00,
         1.08261482e+00,  1.45123644e+00,  8.27017231e-01,
         1.36448709e+00,  4.34506599e-01,  8.81386654e-01,
         8.41462937e-01,  1.11996816e+00,  1.65224657e+00,
         5.55218939e-01,  6.94395477e-02,  3.78259800e-01,
         5.54882370e-01,  9.01287130e-01,  1.49273763e+00,
         4.97614080e-01,  1.00612053e+00,  8.83246589e-01,
         1.35483910e+00,  6.80170908e-01,  9.74166565e-01,
         1.06269953e+00,  7.32865320e-01,  1.11543783e+00,
         1.18804100e+00,  1.43514259e+00,  1.10208341e+00,
         9.39306098e-01,  1.58466577e+00,  9.73248052e-01,
        -8.43491713e-01,  1.45149040e+00,  7.79339160e-01,
         7.92768036e-01,  8.82753138e-01,  7.31424128e-01,
         8.27017255e-01,  9.15793730e-01,  1.05955768e+00,
         7.20077374e-01,  7.97320716e-01,  1.31749866e+00,
         8.40198866e-01,  1.00277899e+00, -1.08145839e+00,
         1.52757988e+00,  1.11989928e+00,  1.25449753e+00,
         1.00923954e+00,  7.69663918e-02,  1.74962202e+00,
         8.58391067e-01,  1.03060994e+00,  1.16775694e+00,
         1.78635199e+00,  8.97126129e-01,  1.15307594e+00,
         8.46289368e-01,  5.60391140e-01,  1.52708629e+00,
         1.13107798e+00,  9.50295197e-01, -2.90886479e-01,
         9.63728933e-01,  9.77340896e-01,  1.43405275e+00,
         7.21211672e-01,  6.06426881e-01,  6.81350209e-01,
         1.41575762e+00,  9.53476450e-01,  1.03533378e+00,
         9.26458133e-01,  1.35712276e+00,  1.49820252e+00,
         9.86607008e-01,  1.17143393e+00,  1.09967601e+00,
         1.89945126e-01,  1.01934736e+00,  6.99018998e-01,
         2.12353102e+00, -1.62645211e-01,  2.21330447e+00,
        -4.53221964e-01,  1.65761525e+00, -2.99921792e+00,
         1.16644075e+00,  1.18961479e+00,  1.45427534e+00,
         1.03004684e+00, -9.03949421e-01,  1.00712383e+00,
         9.31121923e-01,  6.17893385e-01,  1.38713426e+00,
         6.40226923e-01, -4.40181644e-02,  7.73585765e-01,
         1.36834291e+00,  2.02460706e-02,  4.92841602e-01,
         1.20092642e+00,  1.26486883e+00,  2.75418367e-01,
        -7.84585274e-01,  7.78970996e-01,  1.06574204e+00,
         6.14920224e-01,  1.06913891e+00,  1.30310826e+00,
         7.40862535e-03,  3.32358997e-01, -9.31488339e-01,
        -8.17779168e-01,  3.91783500e-01,  1.27360001e+00,
         5.43895111e-01,  1.15393709e+00,  9.60049052e-01,
         7.61123653e-01,  4.86801777e-02],
       [ 1.15586442e+00,  5.00811829e-01, -1.05047706e-01,
         1.33948282e+00, -1.08193831e+00,  7.54636960e-01,
         5.02041806e-01,  1.20143671e+00,  8.41713777e-01,
         9.76488474e-01,  1.43897426e+00,  9.59775536e-01,
         1.13786214e+00,  9.24156256e-01,  1.26411154e+00,
         1.89953641e+00,  9.97658560e-01,  4.37917631e-01,
         1.24390719e+00,  9.82961642e-01,  1.09054830e+00,
         9.26387523e-01,  2.80631894e-01,  1.09048985e+00,
         1.12554727e+00,  9.47709782e-01,  1.04160584e+00,
         1.53889961e+00,  3.83237560e-01,  1.03967449e+00,
         1.08898773e+00,  9.74455101e-01,  8.93658155e-01,
         9.72457881e-01, -1.05047706e-01,  8.90291416e-01,
         7.25764958e-01,  9.82979361e-01,  1.44082109e+00,
         7.59144420e-01,  4.03664564e-01,  1.18556735e+00,
         1.32632946e+00,  3.49141595e-01,  1.21893708e+00,
         1.11843552e+00,  8.42890277e-01,  1.28791369e+00,
         1.41697445e+00,  7.26605861e-01,  8.18568893e-01,
         1.11400051e+00,  8.95348343e-01,  1.40996726e+00,
        -6.24658911e-01,  1.35355792e+00,  1.07676922e+00,
         1.08416149e+00,  1.19246005e+00,  1.52086737e-01,
         9.84439577e-01,  8.64495089e-01,  6.37874764e-01,
         8.68716080e-01,  1.10835285e+00,  1.06457369e+00,
         9.55821656e-01,  1.14318518e+00, -1.65980641e+00,
         1.47306407e+00,  1.23971741e+00,  1.24620857e+00,
         1.09464721e+00,  3.01373596e-02,  1.53966029e+00,
         9.65369709e-01,  1.11451301e+00,  1.31773734e+00,
        -5.38606386e-01,  1.13630507e+00,  1.01355221e+00,
         5.11546229e-01,  1.06962624e+00,  9.61354867e-01,
         1.20365321e+00,  9.86707502e-01, -4.69433391e-02,
         1.11400015e+00,  1.17652554e+00,  1.46040255e+00,
         8.01434527e-01,  7.43120698e-01,  1.05547135e+00,
         1.41061479e+00,  1.50678634e+00,  1.34989339e+00,
         1.01498394e+00,  8.05332019e-01,  8.60143274e-01,
         1.00380452e+00,  9.29290951e-01,  3.79858321e-01,
        -2.50973443e-01,  9.43140020e-01, -2.41128545e-02,
         1.49866365e+00, -3.69508622e-01,  4.71077811e-01,
        -1.22850675e+00, -1.67405350e+00,  3.45999956e-01,
         9.66276994e-01,  3.08361228e-01,  1.91954305e+00,
         1.23973840e+00, -1.46277850e+00,  1.01615914e+00,
        -9.50878632e-01, -3.86395931e+00,  1.58180534e+00,
         1.62562905e+00, -2.48050858e-01, -2.45593472e-01,
         1.44692184e+00, -1.03890616e+00,  3.73698490e-01,
         7.62371668e-01, -4.03625700e-01, -1.86331306e+00,
        -2.89213942e+00,  1.14306982e+00,  9.13851763e-01,
        -2.60012259e+00,  9.65355650e-01, -2.99380348e+00,
         1.63787479e-03,  4.82143579e-01, -6.97171701e-01,
        -1.16996658e+00,  1.29462403e+00,  1.11579844e+00,
         9.30270643e-01,  7.99138800e-01,  1.98926640e-02,
         9.63545850e-01,  4.52319402e-01],
       [ 1.54598833e+00,  4.93197023e-01, -2.13082001e-02,
         9.46022539e-01, -1.42897695e+00,  4.54871981e-01,
         6.59488562e-01,  9.89646389e-01,  7.72059325e-01,
        -9.13131268e-01,  1.24755932e+00,  1.24760038e+00,
         1.12343262e+00,  1.19304496e+00,  1.40961112e+00,
         4.31281673e-01,  6.25366782e-01, -7.65590265e-02,
         1.07761721e+00,  8.95053808e-01,  1.27692970e+00,
         1.12295094e+00,  8.79329393e-01,  9.83915901e-01,
         1.46310954e+00,  8.71856410e-01,  1.46636719e+00,
         1.68491373e+00,  5.06640946e-01,  1.45286891e+00,
         1.35254781e+00,  1.14922894e+00,  8.90025506e-01,
         1.13306943e+00, -2.13082001e-02,  2.02202101e+00,
         9.95391078e-01,  1.22195497e+00,  1.53038001e+00,
         1.59206499e+00,  8.37267689e-01,  1.56621818e+00,
         1.45628305e+00,  1.04718167e+00,  1.51531011e+00,
         1.28270566e+00,  8.61901943e-01,  1.37120629e+00,
         1.45762887e+00,  4.00895865e-01,  9.51078239e-01,
         1.33616797e+00,  8.64904666e-01,  1.59962895e+00,
        -2.79446039e-01,  1.33287156e+00,  1.56789506e+00,
         1.51896922e+00,  1.50360923e+00,  1.71082105e-01,
         1.02023398e+00,  1.17766282e+00,  9.33706194e-01,
         1.28549009e+00,  1.41661674e+00,  7.58354354e-01,
         9.12371053e-01,  1.49788218e+00,  4.87692312e-01,
         1.30693839e+00,  1.52524321e+00,  1.10354819e+00,
         1.19243818e+00, -1.80527815e-02,  4.00933402e-01,
         1.28501004e+00,  9.63200628e-01,  1.36187873e+00,
        -8.14336743e-01, -6.03830704e-01,  6.34821461e-01,
        -5.50634939e-02,  1.07702798e+00,  8.64990790e-01,
         1.32512296e+00,  1.19449653e+00,  5.86335486e-01,
         9.73367766e-01,  1.39410559e+00,  1.14965671e+00,
         1.09432428e+00,  1.04491699e+00,  1.54994058e+00,
         1.21608147e+00,  1.68971532e+00,  1.54977625e+00,
         9.49290581e-01,  2.63394683e-01,  4.08092477e-01,
         1.44234111e+00,  1.88508311e+00, -9.81390018e-02,
        -3.97946299e-01,  9.63923840e-01,  2.38152052e-01,
         1.18922573e+00, -4.15948999e-01,  3.53924007e-01,
        -1.85417517e+00,  1.25784518e+00,  3.64761495e-01,
         1.04967856e+00, -1.00306510e-01,  1.77127636e+00,
         1.49181992e+00, -1.09630602e+00,  1.81870564e+00,
         7.64112769e-02, -1.33276605e-01,  1.25697939e+00,
         2.77453791e+00,  7.58853066e-01,  1.49062836e+00,
         6.05813023e-01,  2.79461816e-01,  1.17135454e+00,
        -4.07746046e-01,  1.92767360e-01, -1.02247411e+00,
         5.77540626e-01,  1.52793095e+00,  1.19068662e+00,
        -1.91128096e-01,  5.71762535e-01,  3.33289953e-01,
         4.77784201e-03,  4.30963267e-01,  1.53840031e+00,
        -1.62971463e-01,  1.85440948e+00,  8.76696228e-01,
         1.14993229e+00,  1.06080254e+00,  1.03942251e+00,
         1.28791489e+00,  6.39446807e-01]]]
data = np.array(data)
def make_preds(model, input_data, std=STD, mean=MEAN):
  forecast = model.predict(input_data)
  preds = tf.squeeze(forecast)
  preds_destandardized = preds * STD + MEAN

  return preds_destandardized

pred = make_preds(model, np.array(data))

1/1 [==============================] - 0s 24ms/step


In [ ]:
pred

<tf.Tensor: shape=(146,), dtype=float32, numpy=
array([ 4945.609 ,  4731.726 ,  3436.904 ,  4394.3774,  5198.4536,
        3906.2554,  4817.0527,  7001.192 ,  5717.455 ,  8645.529 ,
        4706.1445,  5161.2744,  5669.9893,  3496.1262,  5359.0693,
        6099.627 ,  5629.12  ,  6918.5674,  5176.324 ,  8618.036 ,
        8931.737 ,  4158.9956,  3606.5203,  5779.6133,  4802.78  ,
        7644.167 ,  3366.0378,  9849.598 ,  4405.8213,  5832.6396,
        6669.304 ,  4582.362 ,  3733.2   ,  7925.7046,  3465.7495,
        3193.9436,  1974.992 ,  7149.658 ,  3454.0493,  2377.1401,
        4692.5737,  3409.8914,  7199.757 ,  4262.8247,  4975.4756,
        5002.5723,  4466.3613,  4311.863 ,  5383.742 ,  5155.327 ,
        6157.1255,  5451.3706,  7267.6074,  6379.819 ,  3023.5615,
        6072.204 ,  3773.6592,  4724.09  ,  4981.8955,  6763.947 ,
        7282.5166,  6091.329 ,  4560.828 ,  2530.7495,  4587.4517,
        5577.2295,  4315.6265,  6426.34  ,  4664.161 ,  3858.0352,
        6784.8